# ...не забыть сказать пару слов о выпуклой оптимизации

# Введение в Discrete Choice Models

### Логистическая регрессия лежит в пересечении двух разных филдов: первый из них это модели классификации (CS) а вторая это модели дискретного выбора (Econ). Мы поговорим сегодня про второй.

Агент выбирает между двумя товарами 1,2 и дефолтным 0, и его полезность выглядит следующим образом
- $U_0 = V_0(X,Z) + \varepsilon_0$ если он ничего не купил, $V_0(X,Z) = 0$
- $U_1 = V_1(X,Z) + \varepsilon_1$ если он купил первый, $V_1(X,Z) = \alpha' Z + \beta' X$
- $U_2 = V_2(X,Z) + \varepsilon_2$ если он купил второй, $V_2(X,Z) = \alpha' Z + \beta' X$

где $Z$ это характеристики потребителя (доход) а $X$ характеристики товаров (цены)

здесь $\varepsilon_i$ это i.i.d. ошибка с необычным распределением Extreme Value, такая что $\varepsilon_i - \varepsilon_j$  получается распределена логистически

### Вероятности купить какой либо товар выражаются легко

- $P_0 = Prob(U_0 = \max U_i) = Prob(\varepsilon_j - \varepsilon_0 < V_0 - V_j, j \neq 0) = e^{V_0}/(e^{V_0} + e^{V_1} + e^{V_2})$
- $P_1 = Prob(U_1 = \max U_i) = Prob(\varepsilon_j - \varepsilon_1 < V_1 - V_j, j \neq 1) = e^{V_1}/(e^{V_0} + e^{V_1} + e^{V_2})$
- $P_2 = Prob(U_2 = \max U_i) = Prob(\varepsilon_j - \varepsilon_2 < V_2 - V_j, j \neq 2) = e^{V_2}/(e^{V_0} + e^{V_1} + e^{V_2})$

### Заметим, что вероятности складываются в единичку

### Это и есть лоджит: $$ \text{Доля } j = \frac{e^{\alpha' \text{дем.} + \beta' p_j + \gamma' p_i}}{1 + e^{\alpha' \text{дем.}+ \beta' p_j + \gamma' p_i} + e^{\alpha' \text{дем.}+ \beta' p_i + \gamma' p_j}}$$
### Из него можно вытащить, например, эластичности потребления по ценам и бюджету, в среднем

### Логистическая регрессия настолько интуитивна и удобна, что было придумано большое количество интересных обобщений
- ordered logit
- nested logit
- random coefficient logit
- latent group logit

### Это все можно прочитать в книжке Train, Discrete Choice Methods with Simulation, 2003 или 2009

## 1. Ordered Logit 
### это почти лоджит, в котором альтернативы строго упорядочены, скажем, рассмотрим задачу выбора уровня образования. На самом деле, это совсем не лоджит, но выглядит похоже.
$$ U = V(X,Z) + \varepsilon$$
где $X$ это характеристики агента, а $Z$ это характеристики района/школы, а $\varepsilon$ логистическая ошибка

- при полезности $U<r_1$ человек идет в школу $\to$ товар 0
- при полезности $r_1<U<r_2$ человек идет в бакалавриат $\to$ товар 1
- при полезности $r_2<U<r_3$ человек идет в магистратуру $\to$ товар 2
- при полезности $r_3>U$ человек делает PhD $\to$ товар 3

Наша задача оценить параметры $r_1, r_2, r_3$ и остальные коэффициенты

### Все что надо сделать это честно вывести доли этих "товаров", 
$$ \text{Доля товара } 1 = Pr(r_1 < U < r_2) = Pr(U < r_2) - Pr(U < r_1)=$$ $$= Pr(\varepsilon < r_2 - V(X,Z)) - Pr(\varepsilon < r_1 - V(X,Z)) = \frac{1}{1+\exp(V(X,Z)-r_2)} - \frac{1}{1+\exp(V(X,Z)-r_1)}$$

### Затем можно честно вывести лог лайклихуд
$$ \mathcal{LL} = \sum_{y_i=0} \log[\frac{1}{1+\exp(V(x_i,z_i)-r_1)}]+ \sum_{y_i=1}\log[\frac{1}{1+\exp(V(x_i,z_i)-r_2)}-\frac{1}{1+\exp(V(x_i,z_i)-r_1)}] + \ldots + \sum_{y_i = 3}\log [1-\frac{1}{1+\exp(V(x_i,z_i)-r_1)}]$$

### Казалось бы, делаем линейную модель $V(X,Z) = \alpha X + \beta Z$ и вперед
### Важный вопрос, является ли эта штука вогнутой по $\alpha, \beta, r$? Я долго пытался проверить вогнутость: 
$$ \log[\exp(\alpha' x_i-r_1)-\exp(\alpha' x_i-r_2)]$$
### для этого куска, и похоже он действительно вогнутый.

## 2. Latent group Logit
### Иногда мы хотим отразить скореллированность коэффициентов. Скажем, у части населения оба коэффицинта $\beta_1$ и $\beta_2$ маленькие а у части населения $\beta_1$ и $\beta_2$ большие. Сами эти группы мы не видим в данных, но знаем что они есть. Тогда мы создаем две пачки коэффициентов $\beta^1$ и $\beta^2$ и еще дополнительно параметр $\rho$ отвечающий за долю первой группы. Моделируем так, что агент случайно получает свою группу а потом идет с соответствующими коэффициентами покупать.

$$\mathcal{L}=\prod G(x)^{y_i} \cdot (1-G(x))^{1-y_i}, \quad G(x) = \rho F(\beta^1 x_i) + (1-\rho)F(\beta^2 x_i)$$

### Является ли $\mathcal{LL}$ тут вогнутой по $\beta_1, \beta_2, \rho$? У меня не хватило терпения вычислить, но шанс есть. Надо читать Kamakura & Russell (1989) and Chintaguntaet al.(1991)

## 3. Random Coefficients Logit
### Иногда двух или нескольких групп не достаточно, мы хотим непрерывно коррелировать коэффициенты.

$$\mathcal{L}=\prod G(x)^{y_i} \cdot (1-G(x))^{1-y_i}, \quad G(x) = \int F(\beta x_i) d H(\beta)$$
где $H(\beta)$ это какое то распределение с неизвестным средним/ковариационной матрицей. Оптимизация будет по параметрам этого распределения.

### Является ли $\mathcal{LL}$ тут вогнутой? Интеграл это сумма, так что все упирается в плотность распределения $H$. Опять надо читать Revelt  &  Train  (1998),  Mehndiratta  (1996),  and  Ben-Akiva& Bolduc (1996)

### Более того, $G(x)$ не обязательно выписывается явно, тогда придется считать интеграл численно. Скажем, наш коэффициент $\beta$ распределен нормально со средним $\gamma$ и дисперсией $\alpha^2$:
$$ \beta = \alpha \mathcal{N}(0,1) + \gamma, \quad \mathcal{LL} \to \max_{\alpha, \gamma}$$

### Для этого сначала симулируется (снаружи цикла) большое количество нормальных случайных величин, а затем при помощи личейного преобразования получаются (внутри цикла) какие надо и по ним суммируется интеграл. Обычно, хватает 1000 симуляций, если параметров не слишком много.

## 4. Nested Logit
### Иногда какие то группы альтернатив хочется логически связать, чтобы подчеркнуть, что они субституты. Тогда выделяются переменные $X$, с коэффициентом $\beta$, характеризующие товар и переменные $Z$ с коэффициентом $\gamma$, характеризующие группу. Полезность от группы и от товара как бы суммируется: 

$$ U_{i} = V_i + \varepsilon = \beta' X_i + \gamma' Z_g + \varepsilon$$ 

### Более того, ошибки в группах хитрым образом скореллированы, как если бы: $\varepsilon = \varepsilon_i + \lambda_g \varepsilon_g$, но я не помню настоящую формулу и не смог ее найти. Да это и неважно
### Без ограничения общности, агент сначала выбирает в каждой группе лучший товар, а потом группу
$$\mathcal{L}=\prod G(x)^{y_i} \cdot (1-G(x))^{1-y_i}, \quad G(x) = Pr(\text{купить товар}|\text{группа товаров})Pr(\text{выбрать группу})$$
### Пусть всего $K$ груп и коэффициенты $\lambda_1, \ldots, \lambda_K$. Тогда формула для доли получается такая
$$ \text{Доля товара $i$ из группы $g_k$ равна} \quad  \frac{\exp(V_i/\lambda_k)\cdot (\sum_{j \in g_k} \exp(V_j/\lambda_k))^{\lambda_k-1}}{(\sum_{j \in g_1} \exp(V_j /\lambda_1))^{\lambda_1} + \ldots + (\sum_{j \in g_K}  \exp(V_j /\lambda_K))^{\lambda_K}}$$

### Глобальной выпуклости нет, это подтвержденный факт

## P.S. Все это можно делать с регуляризацией. 
### Более того, это настоятельно рекомендуется в случаях когда задача не выпуклая, чтобы удержать коэффициенты от резкого возрастания или убывания, что может больно ударить с экспонентой